In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray
from matplotlib import pyplot as plt
import codebase

In [ ]:
res_shp = codebase.load_data.load_GRanD()

In [ ]:
from codebase import volume_pipeline

In [ ]:
# Step 1 wrapper function works
dem_DA , fw_DA = volume_pipeline.create_aligned_DEM_CYGNSS_subsets('Indirasagar',res_shp)

In [ ]:
# Step 2 function works
fw_diff_DA = volume_pipeline.difference_over_time(fw_DA)

In [ ]:
# Step 3 single timestep and vectorized work.
change_cond_DA = volume_pipeline.decide_expansion_or_shrinkage_vectorize(fw_diff_DA)

# Testing out vectorization of Step 3 (label as expand/shrink for each timestep)
test_vectorized = xr.apply_ufunc(volume_pipeline.decide_expansion_or_shrinkage_timestep,
                                 fw_diff_DA,input_core_dims=[['lat','lon']],vectorize=True)

In [ ]:
# Step 4 wrapper function works
from scipy.stats import norm
change_cond_time0 = change_cond_DA.isel(time=0)
fw_diff_time0 = fw_diff_DA.isel(time=0)
fw_time0 = fw_DA.isel(time=0)
start_params = volume_pipeline.fit_DEM_distribution_from_conditional_area(dem_DA, fw_time0, 1, norm)
change_params = volume_pipeline.fit_DEM_distribution_from_conditional_area(dem_DA, fw_diff_time0, change_cond_time0, norm)
print('params of start timestep:',start_params)
print('params of change in area:',change_params)

In [ ]:
## Could not vectorize the Step 4 function(s). Skipped vectorization and made for loop functions :(

# from xarray import DataArray
# def grab_DEM_of_conditional_area(
#     dem_DA: DataArray, cond_DA: DataArray, cond: int = 1
# ) -> DataArray:
#     print(type(dem_DA), type(cond_DA) , cond)
#     print(dem_DA.shape)
#     dem_cond_area = dem_DA.where(cond_DA == cond)
#     return dem_cond_area


# xr.apply_ufunc(grab_DEM_of_conditional_area,
#                fw_DA,input_core_dims=[['lat','lon']],kwargs={'dem_DA':dem_DA},
#               vectorize=True, output_core_dims=['time'])

# xr.apply_ufunc(volume_pipeline.fit_DEM_distribution_from_conditional_area,
#                dem_DA,fw_DA[:-1,:,:],change_cond_DA,norm,
#               input_core_dims=[['lat','lon'],['lat','lon'],[],[]],
#               vectorize=True, join='right')

In [ ]:
test_params = volume_pipeline.loop_through_time_series_to_get_fit_params(dem_DA,fw_diff_DA,change_cond_DA,norm)

In [ ]:
test_heights = volume_pipeline.calculate_height_time_series_from_start_and_change_in_DEM(dem_DA,fw_DA,fw_diff_DA,change_cond_DA)

In [ ]:
test_heights

In [ ]:
fw_diff_DA.isel(time=-1).plot(cmap='RdBu')

In [ ]:
test_params[0]

In [ ]:
def calculate_rough_area(input_DA):
    """
    Placeholder function while I get the pipeline up and running.
    Should replace with function(s) project into 2D space.
    """
    pixel_count = input_DA.sum()
    area_deg2 = 0.01 *0.01 * pixel_count
    return area_deg2

In [ ]:
volume_pipeline.calculate_rough_area_vectorize(fw_DA)

In [ ]:
test_reproj = fw_DA.rio.reproject("ESRI:54017")